In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classes = base.iloc[:1400000, 20:28].values
classes = np.reshape(classes, (2000, 700, 8))
print(classes.shape)

(2000, 700, 20)
(2000, 700, 8)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNGRU, Bidirectional, Masking

In [6]:
def criarRede():
    model = Sequential()
  
    #model.add(Masking(mask_value = 0, input_shape = (700, 20)))
  
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Dense(8, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [7]:
accu = []
precisions = []
recalls = []
f1 = []

In [8]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 100, verbose = 1, batch_size = 32)
  
    predicted = model.predict(x_test)
  
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], 8))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 8))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste

    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accu.append(balanced_accuracy_score(y_test, predicted)) 
    precisions.append(precision_score(y_test, predicted, average = 'weighted'))
    recalls.append(recall_score(y_test, predicted, average = 'weighted'))
    f1.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede()
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classes[train_index],
                           previsores[test_index], classes[test_index])

W0822 10:32:57.561510  8048 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0822 10:32:57.565528  8048 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0822 10:32:57.566497  8048 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   0    1    2 ... 1996 1998 1999] TEST: [   4   16   26   27   29   33   38   41   43   49   52   55   59   66
   68   72   77   79   83   92   93   97  112  115  117  124  126  138
  140  143  145  157  159  164  166  173  176  178  187  193  199  203
  204  209  210  214  215  221  223  225  226  245  246  251  255  258
  262  265  267  269  273  274  280  288  295  297  313  314  315  317
  319  320  325  333  334  336  342  344  345  347  352  360  362  369
  373  380  388  394  395  401  413  423  433  441  445  446  452  461
  462  467  470  473  474  480  487  492  497  499  521  522  523  529
  533  543  548  554  561  572  582  586  589  591  595  596  600  603
  604  605  610  615  622  629  630  636  638  639  644  646  648  655
  656  661  668  669  676  680  681  696  710  714  716  718  722  729
  740  743  747  748  752  757  764  765  798  810  823  827  828  845
  851  852  870  871  872  877  878  896  899  907  909  911  914  927
  929  946  952  953  957  9

W0822 10:32:58.453149  8048 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.5900 - acc: 0.1589
Epoch 2/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.5331 - acc: 0.1722
Epoch 3/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.5222 - acc: 0.1775
Epoch 4/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.5129 - acc: 0.1813
Epoch 5/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.5101 - acc: 0.1823
Epoch 6/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.5055 - acc: 0.1844
Epoch 7/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.5010 - acc: 0.1864
Epoch 8/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.4974 - acc: 0.1881
Epoch 9/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.4931 - acc: 0.1897
Epoch 10/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.4925 - acc: 0.190

1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3182 - acc: 0.2515
Epoch 83/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3166 - acc: 0.2524
Epoch 84/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3159 - acc: 0.2521
Epoch 85/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3136 - acc: 0.2530
Epoch 86/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3128 - acc: 0.2534
Epoch 87/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3117 - acc: 0.2537
Epoch 88/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3107 - acc: 0.2540
Epoch 89/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3099 - acc: 0.2544
Epoch 90/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3082 - acc: 0.2550
Epoch 91/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3076 - acc: 0.2551
Epo

...

c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1181
           1       0.62      0.70      0.66     19700
           2       0.38      0.23      0.29      3692
           3       0.74      0.83      0.78     32898
           4       0.49      0.20      0.29       718
           5       0.47      0.53      0.50     19573
           6       0.38      0.12      0.18      8532
           7       0.41      0.40      0.40     10898

    accuracy                           0.59     97192
   macro avg       0.43      0.38      0.39     97192
weighted avg       0.57      0.59      0.57     97192

Acurácia
0.3764314093914398
Precisao
0.5664396111581879
Recall
0.5938760391801794
F1
0.5712182879951612
[[    0   300    23   154     1   532    42   129]
 [    0 13778   158  2114    13  2691   190   756]
 [    0   464   861   920     6   827    89   525]
 [    0  1607   345 27159    83  2107   176  1421]
 [    0    51     3   402   147    68    

1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3617 - acc: 0.2339
Epoch 51/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3599 - acc: 0.2342
Epoch 52/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3582 - acc: 0.2347
Epoch 53/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3559 - acc: 0.2355
Epoch 54/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3553 - acc: 0.2358
Epoch 55/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3526 - acc: 0.2366
Epoch 56/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3503 - acc: 0.2374
Epoch 57/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3485 - acc: 0.2377
Epoch 58/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3451 - acc: 0.2392
Epoch 59/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3437 - acc: 0.2395
Epo

Epoch 1/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.5831 - acc: 0.1644
Epoch 2/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.5291 - acc: 0.1711
Epoch 3/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.5180 - acc: 0.1764
Epoch 4/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.5092 - acc: 0.1802
Epoch 5/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.5050 - acc: 0.1823
Epoch 6/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.5004 - acc: 0.1844
Epoch 7/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.4972 - acc: 0.1856
Epoch 8/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.4936 - acc: 0.1872
Epoch 9/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.4913 - acc: 0.1882
Epoch 10/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.4878 - acc: 0.1896

1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3158 - acc: 0.2499
Epoch 83/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3146 - acc: 0.2502
Epoch 84/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3139 - acc: 0.2505
Epoch 85/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3127 - acc: 0.2510
Epoch 86/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3107 - acc: 0.2513
Epoch 87/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3103 - acc: 0.2517
Epoch 88/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3097 - acc: 0.2518
Epoch 89/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3091 - acc: 0.2520
Epoch 90/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3077 - acc: 0.2525
Epoch 91/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3071 - acc: 0.2527
Epo

In [9]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_12 (Bidirectio (None, 700, 200)          97600     
_________________________________________________________________
dropout_12 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_13 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_14 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
dense_4 (Dense)              (None, 700, 8)           

In [10]:
print('Acurácias total')
print(accu)
a = np.array(accu)
print(a.mean())
print('Precision total')
print(precisions)
p = np.array(precisions)
print(p.mean())
print('Recalls total')
print(recalls)
r = np.array(recalls)
print(r.mean())
print('F1 total')
print(f1)
f = np.array(f1)
print(f.mean())

Acurácias total
[0.35915306700300176, 0.3657686597248555, 0.3730990359060886, 0.37638366504933796, 0.36126651444129454]
0.36713418842491563
Precision total
[0.5358692747285666, 0.539195778418488, 0.5569088715949322, 0.5490842678202248, 0.5354863437177183]
0.543308907255986
Recalls total
[0.5482441618136844, 0.5525903724410886, 0.5657434373839448, 0.5568281331483504, 0.5502227044312471]
0.554725761843663
F1 total
[0.531212445417416, 0.5355347926101575, 0.5486665733108644, 0.5431847405399131, 0.5336637122813451]
0.5384524528319392
